# Multi Environment Json Pattern

This pattern allows you to manage configuration for multiple environments.

In modern software development, managing configurations across multiple environments has become a crucial aspect of maintaining robust applications. This document introduce a best practice to manage schema definition, data initialization, config deployment and config usage.

## Solution Overview

1. Utilize dataclasses to define the configuration schema, providing an object-oriented interface to access configuration values. This not only enables basic validation but also helps avoid typographical errors and offers auto-complete functionality within Integrated Development Environments (IDEs).
2. Separating the schema declaration from the data initialization enables a flexible strategy to load the config data from different sources in different environments. For example, you can load the config data from a local file in the development environment and retrieve the config data from AWS Parameter Store in the production environment.
3. Manage the source-of-truth of the config data in Git for non-sensitive data, enabling version control of the config data.
4. Leverage the inheritance hierarchy pattern to set global default values and be able to override them in specific environments.
5. Provide two options to deploy the config data to [AWS Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) or [AWS S3](https://aws.amazon.com/s3/), with encryption-at-transit and encryption-at-rest enabled by default.
6. Isolate the deployment of configurations based on the environment, ensuring that each environment can only access its specific config data and cannot access the configuration of other environments. Additionally, maintain a versioned copy of the all-in-one configuration deployment as a backup for disaster recovery or compliance requirements.
7. Separate static config values from dynamic values, enabling developers to customize the logic for generating dynamic values.



4. Your application runtime can read the config from one of the config storage options above.
5. When you deploy your application, you should store the ``parameter_name`` or ``s3path_config`` information to the environment variable or a static file. So your application can use this information to read the config data from the config storage.

## Config Json File Format

In object-oriented programming, the inheritance hierarchy is a pattern where child objects inherit attributes and methods from parent objects. Similarly, in configuration management, the global configuration often acts as the default value, allowing for the possibility of overriding specific values in environment-specific configurations. Please take a look at the following example of non-sensitive config and secret config JSON file. We will explain it later.

In [1]:
import os
import json
from pathlib import Path
from rich import print as rprint

def jprint(data: dict):
    rprint(json.dumps(data, indent=4))

dir_here = Path(os.getcwd()).absolute()
path_config = dir_here.joinpath("config.json")
path_secret_config = dir_here.joinpath("secret_config.json")

In [2]:
rprint(path_config.read_text())

{
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user"
    },
    "int": {
        "username": "int.user"
    },
    "prod": {
        "username": "prod.user"
    }
}

In [3]:
rprint(path_secret_config.read_text())

{
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "int": {
        "password": "int.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

There's a meta field ``_shared`` in the root level of the config file. It is a powerful inheritance hierarchy mechanism to specify config values. The ``_shared`` field is a key value pairs of JSON path notation and it's value. For example:

In [7]:
from config_patterns.patterns.hierarchy import SHARED, apply_shared_value

data = {
    SHARED: {
        "*.name": "alice",
        "*.contact.email": "alice@email.com",
    },
    "dev": {
        "contact": {},
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        },
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "*.name": "alice",
        "*.contact.email": "alice@email.com"
    },
    "dev": {
        "contact": {}
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        }
    }
}

after:


{
    "dev": {
        "contact": {
            "email": "alice@email.com"
        },
        "name": "alice"
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        }
    }
}

This mechanism is very powerful, it works with list of dict too.

In [8]:
data = {
    SHARED: {
        "*.databases.port": 5432
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com"},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com"},
        ],
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "*.databases.port": 5432
    },
    "dev": {
        "databases": [
            {
                "host": "db1.com"
            },
            {
                "host": "db2.com"
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com"
            },
            {
                "host": "db4.com"
            }
        ]
    }
}

after:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 5432
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 5432
            },
            {
                "host": "db4.com",
                "port": 5432
            }
        ]
    }
}

You can also specify default different value for different environment.

In [6]:
data = {
    SHARED: {
        "dev.databases.port": 5432,
        "prod.databases.port": 3306,
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com", "port": 0},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com", "port": 1},
        ],
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "dev.databases.port": 5432,
        "prod.databases.port": 3306
    },
    "dev": {
        "databases": [
            {
                "host": "db1.com"
            },
            {
                "host": "db2.com",
                "port": 0
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com"
            },
            {
                "host": "db4.com",
                "port": 1
            }
        ]
    }
}

after:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 0
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 3306
            },
            {
                "host": "db4.com",
                "port": 1
            }
        ]
    }
}

a## Sample Usage

### Declare Your Config Schema

In software engineer best practice, declaration and the usage of a Data Model should be separated. Below is the ``config_define.py`` file that defines three things:

1. enumerate all environments you want to use in your project.
2. declare the per environment config data model.
3. subclass from the BaseConfig, this is your main config object.

In [1]:
# content of config_define.py
# -*- coding: utf-8 -*-

import typing as T
import dataclasses

from config_patterns.patterns.multi_env_json import (
    BaseEnvEnum,
    BaseEnv,
    BaseConfig,
)


class EnvEnum(BaseEnvEnum):
    dev = "dev" # development
    int = "int" # integration test
    prod = "prod" # production


@dataclasses.dataclass
class Env(BaseEnv):
    username: T.Optional[str] = dataclasses.field(default=None)
    password: T.Optional[str] = dataclasses.field(default=None)


@dataclasses.dataclass
class Config(BaseConfig):
    @property
    def dev(self) -> Env:
        return self.get_env(EnvEnum.dev)

    @property
    def int(self) -> Env:
        return self.get_env(EnvEnum.int)

    @property
    def prod(self) -> Env:
        return self.get_env(EnvEnum.prod)

    @classmethod
    def get_current_env(cls) -> str:
        return EnvEnum.dev.value

    @property
    def env(self) -> Env:
        return self.get_env(self.get_current_env())

    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)


### Read From Local File and Deploy to Config Storage

As the project admin, you need to decide what value to put in the config. So you created two config files ``config.json`` and ``secret_config.json``. You could check in the ``config.json`` to the Git so everyone can see it. But keep the ``secret_config.json`` private, and only give access to people really need it.

In [2]:
# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "project_name": "my_project"
    },
    "dev": {
        "username": "dev.user"
    },
    "int": {
        "username": "int.user"
    },
    "prod": {
        "username": "prod.user"
    }
}

{'_shared': {'project_name': 'my_project'},
 'dev': {'username': 'dev.user'},
 'int': {'username': 'int.user'},
 'prod': {'username': 'prod.user'}}

In [3]:
# content of secret-config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "int": {
        "password": "int.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

{'_shared': {},
 'dev': {'password': 'dev.password'},
 'int': {'password': 'int.password'},
 'prod': {'password': 'prod.password'}}

Then you can create the config object and deploy it to config storage.

In [4]:
# content of config_deploy.py
# -*- coding: utf-8 -*-

# import from the config_define.py
from config_define import EnvEnum, Env, Config

import os
from pathlib import Path
from boto_session_manager import BotoSesManager
from rich import print as rprint

# Read config from local file
dir_here = Path(os.getcwd())
path_config = str(dir_here.joinpath("config.json"))
path_secret_config = str(dir_here.joinpath("secret_config.json"))


config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)
rprint(config)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user'},
        'int': {'username': 'int.user'},
        'prod': {'username': 'prod.user'}
    },
    secret_data={
        '_shared': {},
        'dev': {'password': 'dev.password'},
        'int': {'password': 'int.password'},
        'prod': {'password': 'prod.password'}
    },
    Env=<class 'config_define.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    _merged_data={
        'dev': {'username': 'dev.user', 'project_name': 'my_project'},
        'int': {'username': 'int.user', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user', 'project_name': 'my_project'}
    },
    _merged_secret_data={
        'dev': {'password': 'dev.password'},
        'int': {'password': 'int.password'},
        'prod': {'password': 'prod.password'}
    },
    _merged={
        'dev': {'username': 'dev.user', 'project_name': 'my_project', 'password': 'dev.password'},
        'int': {'username': 'int.user', 'project_name': 'my_project', 'password': 'int.password'},
        'prod': {'username': 'prod.user', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In [12]:
# Deploy config to AWS Parameter Store
bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")
s3dir_config = "s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"


deployment_list = config.deploy(
    bsm=bsm,
    parameter_with_encryption=True,
)
rprint(deployment_list)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter '/app/my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/app/my_project/description?region=us-east-1&tab=Table
| parameter data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.27 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter '/app/my_project-dev' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/app/my_project-dev/description?region=us-east-1&tab=Table
| parameter data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.13 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter '/app/my_project-int' ...
| preview at: https://us-east-1.cons

[
    ConfigDeployment(
        parameter_name='/app/my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=None,
        deletion=None
    )
]

In [13]:
# Deploy config to AWS S3 Store
deployment_list = config.deploy(
    bsm=bsm,
    s3dir_config=s3dir_config,
)
rprint(deployment_list)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/app/my_project.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/app/my_project.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.14 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/app/my_project-dev.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/app/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.08 se

[
    ConfigDeployment(
        parameter_name='/app/my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=S3Object(
            expiration=None,
            etag='"ac411b52f2670c32151cb4100dd224a7"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=S3Object(
            expiration=None,
            etag='"7163f20371e6da6c6b4e28b807339cf5"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='/app/my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=S3Object(
            expiration=None,
            etag='"9f709ac7c27b0912a37d368db37cd96c"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    )
]

### Use Your Config in Application Code

In your application code, you could create the config object by reading the config storage. Then use the Python config object to access those config values.

In [14]:
# content of config_init.py
# -*- coding: utf-8 -*-

from config_define import EnvEnum, Env, Config

from rich import print as rprint
from boto_session_manager import BotoSesManager

# create boto session manager object for AWS SDK authentication
bsm = BotoSesManager(profile_name="aws_data_lab_sanhe_us_east_1")
parameter_name = "my_project-dev"
s3dir_config = "s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"

You could read the config from AWS Parameter Store. For security reason, assuming that you want to load the "dev" config, you won't be able to access any of the "prod" config from your application runtime.

In [16]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name=parameter_name,
    parameter_with_encryption=True,
)
# rprint(config)
# rprint(config.dev)
# rprint(f"config.dev.username = {config.dev.username!r}")
# rprint(f"config.dev.password = {config.dev.password!r}")

my_project-dev


KeyError: '_shared'

In [18]:
# You can NOT access prod config from dev environment
config.prod

Env(project_name='my_project', env_name='prod', username='prod.user', password='prod.password')

You could also read the config from AWS S3.

In [10]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    s3path_config=f"{s3dir_config}my_project-prod.json",
)
rprint(config)
rprint(config.prod)
rprint(f"config.prod.username = {config.prod.username!r}")
rprint(f"config.prod.password = {config.prod.password!r}")

Config(
    data={'shared': {'project_name': 'my_project'}, 'envs': {'prod': {'username': 'prod.user'}}},
    secret_data={'shared': {}, 'envs': {'prod': {'password': 'prod.password'}}},
    Env=<class 'config_define.Env'>,
    EnvEnum=<enum 'EnvEnum'>
)

Env(project_name='my_project', env_name='prod', username='prod.user', password='prod.password')

config.prod.username = 'prod.user'

config.prod.password = 'prod.password'

## Delete Config from Config Storage

At the end, you can delete all config from config storage.

In [11]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)

In [12]:
config.delete(
    bsm=bsm,
    use_parameter_store=True,
)

delete parameter store for all environment
🗑️ delete SSM Parameter 'my_project' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-dev' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-int' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-int/description?region=us-east-1&tab=Table
done!
🗑️ delete SSM Parameter 'my_project-prod' ...
verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/description?region=us-east-1&tab=Table
done!


In [13]:
config.delete(
    bsm=bsm,
    s3dir_config=s3dir_config,
)

🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project-dev.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project-dev.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project-int.json ...
preview at: https://us-east-1.console.aws.amazon.com/s3/object/669508176277-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project-int.json
done!
🗑️ delete config file s3://669508176277-us-east-1-artifacts/projects/config_patte

## Hierarchy config schema




## Store Sensitive Config Values Separately
